In [17]:
import torch
import torchvision.transforms as transforms
from torch.utils import data
from tiny_imagenet import TinyImageNet

In [18]:
%reload_ext autoreload
%autoreload 2

## 步驟
- 讀取資料集
- 簡單 EDA
    - facets
- 定義目標 / loss function
- 定義模型
- 訓練模型
- 測試模型
- 視覺化 kernels / parameters

## 前處理資料
- 讀取資料
- 轉換（灰階處理、Augementation、Crop）

In [19]:
root = './dataset'

In [20]:
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [21]:
training_set = TinyImageNet(root, 'train', transform=transform)
validation_set = TinyImageNet(root, 'val', transform=transform)

In [22]:
training_set

Dataset TinyImageNet
    Number of datapoints: 100000
    Split: train
    Root Location: ./dataset
    Transforms (if any): Compose(
                             Lambda()
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                         )
    Target Transforms (if any): None

## 訓練模型

In [23]:
params = {'batch_size': 64, 'shuffle': True, 'num_workers': 6}
# params = {'batch_size': 2, 'shuffle': True, 'num_workers': 1}
max_epochs = 100

In [24]:
training_loader = data.DataLoader(training_set, **params)

In [28]:
for epoch in range(1):
    for idx, (data, target) in enumerate(training_loader):
        print(f'{idx}/{len(training_loader)}')

0/1563
1/1563
2/1563
3/1563
4/1563
5/1563
6/1563
7/1563
8/1563
9/1563
10/1563
11/1563


Process Process-25:
Process Process-29:
Traceback (most recent call last):
Process Process-27:
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Process-30:
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <li

RuntimeError: DataLoader worker (pid 34968) exited unexpectedly with exit code 1.

Process Process-28:
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
Traceback (most recent call last):
  File "/Users/leemeng/Documents/Code/tiny-imagenet/tiny_imagenet.py", line 62, in __getitem__
    img = Image.open(file_path)
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/PIL/Image.py", line 2548, in open
    fp = builtins.open(filename, "rb")
Traceback (most recent cal